## Notebook content

This notebook monitors Kubeflow Pipelines (KFP) runs for an AutoML experiment, displays the leaderboard, and optionally downloads a trained model to load and run predictions. It aligns with the **experiment** side of the pipeline (run details, leaderboard, model download).

💡 **Tips:**
- Ensure the KFP connection is configured (e.g. `KF_PIPELINES_SA_TOKEN_PATH`, `BEARER_TOKEN`) so the notebook can list runs and experiments.
- Ensure the S3 connection is configured for the leaderboard and model download (e.g. `AWS_S3_ENDPOINT`, `AWS_S3_BUCKET`).
- Set `experiment_name` and `run_name` to match your pipeline experiment and run.

### Contents
This notebook contains the following parts:

**[Setup](#setup)**  
**[Run AutoML experiment pipeline](#run-automl-experiment-pipeline)**  
**[Monitor KFP run details](#monitor-kfp-run-details)**  
**[Display leaderboard](#display-leaderboard)**  
**[Download trained model](#download-trained-model)**  
**[Load the model](#load-the-model)**  
**[Summary and next steps](#summary-and-next-steps)**

<a id="setup"></a>
## Setup

In [ ]:
import warnings

warnings.filterwarnings("ignore")

<a id="run-automl-experiment-pipeline"></a>
## Run AutoML experiment pipeline

This section shows how to **submit** a new run of the AutoGluon tabular training pipeline. The pipeline is defined in the **pipelines-components** repository:

`pipelines-components/pipelines/training/automl/autogluon_tabular_training_pipeline`

It loads data from S3, trains AutoGluon models, selects the top N, refits them on the full dataset, and produces a leaderboard. You can then use **Monitor KFP run details** to track the run and **Display leaderboard** / **Download trained model** once it has succeeded.

📌 **Action:** Set `pipelines_components_root` to the path of your pipelines-components repo (or set `PIPELINES_COMPONENTS_ROOT`). Ensure the pipeline package and dependencies are installed (e.g. `pip install kfp kfp-components` from pipelines-components).


### Add pipeline package and import

Add the pipelines-components root to `sys.path` so we can import the pipeline function, then compile and submit.


In [ ]:
import os
import sys

# Root of pipelines-components repo (directory that contains the 'pipelines' package)
pipelines_components_root = os.environ.get(
    "PIPELINES_COMPONENTS_ROOT",
    "/path/to/pipelines-components",  # set this or set env PIPELINES_COMPONENTS_ROOT
)
if pipelines_components_root not in sys.path:
    sys.path.insert(0, pipelines_components_root)

from kfp.compiler import Compiler
from pipelines.training.automl.autogluon_tabular_training_pipeline import (
    autogluon_tabular_training_pipeline,
)

### Configuration

Set pipeline parameters (S3 data location, label column, task type, top N models) and the experiment and run names for the new run.


In [ ]:
# Pipeline parameters (match autogluon_tabular_training_pipeline arguments)
pipeline_params = {
    "train_data_secret_name": "s3-credentials",
    "train_data_bucket_name": "my-data-bucket",
    "train_data_file_key": "datasets/train.csv",
    "label_column": "target",
    "task_type": "binary",  # or "multiclass", "regression"
    "top_n": 3,
}

# Experiment name (grouping for runs)
run_experiment_name = "AutoML Titanic"

# Display name for this run
run_display_name = "Titanic survived prediction 0.1.0"

# Compiled pipeline YAML path
pipeline_package_path = "autogluon_tabular_training_pipeline.yaml"


### Compile and submit the pipeline

Compile the pipeline to a YAML file, then create a KFP client (same connection as in **Monitor KFP run details**) and submit a new run.


In [ ]:
compiler = Compiler()
compiler.compile(
    pipeline_func=autogluon_tabular_training_pipeline,
    package_path=pipeline_package_path,
)
print(f"Compiled pipeline to {pipeline_package_path}")

In [ ]:
import kfp

kubeflow_endpoint = os.environ.get("KF_PIPELINES_SA_ENDPOINT", "https://ds-pipeline-dspa-autox.apps.rosa.rhoai-dev.mfe3.p3.openshiftapps.com")
sa_token_path = os.environ.get("KF_PIPELINES_SA_TOKEN_PATH", "/var/run/secrets/kubernetes.io/serviceaccount/token")
sa_ca_cert = os.environ.get("PIPELINES_SSL_SA_CERTS", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")

if "BEARER_TOKEN" in os.environ:
    bearer_token = os.environ["BEARER_TOKEN"]
elif os.path.isfile(sa_token_path):
    with open(sa_token_path) as f:
        bearer_token = f.read().rstrip()
else:
    raise RuntimeError("Set BEARER_TOKEN or KF_PIPELINES_SA_TOKEN_PATH")

ssl_ca_cert = sa_ca_cert if (os.path.isfile(sa_ca_cert) and "svc" in kubeflow_endpoint) else None

run_client = kfp.Client(
    host=kubeflow_endpoint,
    existing_token=bearer_token,
    ssl_ca_cert=ssl_ca_cert,
)

run_result = run_client.create_run_from_pipeline_package(
    pipeline_file=pipeline_package_path,
    arguments=pipeline_params,
    run_name=run_display_name,
    experiment_name=run_experiment_name,
)

submitted_run_id = run_result.run_id
print(f"Submitted run: {submitted_run_id}")
print(f"Use this run_id in Monitor KFP run details or set run_id = '{submitted_run_id}' for leaderboard/download.")

<a id="monitor-kfp-run-details"></a>
## Monitor KFP run details

📌 **Action:** Set `kubeflow_endpoint`, `experiment_name`, and `run_name` to match your KFP pipeline and run. Ensure `KF_PIPELINES_SA_TOKEN_PATH` or `BEARER_TOKEN` is configured.


### initialize kfp client

In [20]:
import os
import kfp

kubeflow_endpoint = "https://ds-pipeline-dspa-autox.apps.rosa.rhoai-dev.mfe3.p3.openshiftapps.com"
print(f"Connecting to kfp: {kubeflow_endpoint}")

sa_token_path = os.environ["KF_PIPELINES_SA_TOKEN_PATH"]
sa_ca_cert = os.environ["PIPELINES_SSL_SA_CERTS"]

if "BEARER_TOKEN" in os.environ:
    print("BEARER_TOKEN provided as environment variable")
    bearer_token = os.environ["BEARER_TOKEN"]
elif os.path.isfile(sa_token_path):
    print("BEARER_TOKEN found as service account token")
    with open(sa_token_path) as f:
        bearer_token = f.read().rstrip()

if os.path.isfile(sa_ca_cert) and "svc" in kubeflow_endpoint:
    print("CA Cert found in container")
    ssl_ca_cert = sa_ca_cert
else:
    ssl_ca_cert = None

client = kfp.Client(
    host=kubeflow_endpoint,
    existing_token=bearer_token,
    ssl_ca_cert=ssl_ca_cert,
)

Connecting to kfp: https://ds-pipeline-dspa-autox.apps.rosa.rhoai-dev.mfe3.p3.openshiftapps.com
BEARER_TOKEN found as service account token


### get pipeline details

In [21]:
pipeline_name = "autogluon-tabular-training-pipeline"
pipeline_id = client.get_pipeline_id(pipeline_name)
component_names = []

for ver in client.list_pipeline_versions(pipeline_id=pipeline_id).to_dict()["pipeline_versions"]:
    components = ver["pipeline_spec"]["pipeline_spec"]["components"]
    for key, value in components.items():
        component_names.append(key.removeprefix("comp-"))

print("components", component_names)

components ['autogluon-models-full-refit', 'automl-data-loader', 'for-loop-1', 'leaderboard-evaluation', 'models-selection', 'train-test-split']


### list experiments

In [22]:
experiment_name = "AutoML Titanic"
experiment_id = None

for exp in client.list_experiments().to_dict()["experiments"]:
    if exp["display_name"] == experiment_name:
        experiment_id = exp["experiment_id"]
    
print("experiment_id", experiment_id)

experiment_id 4894852c-ab35-4f2e-b390-5e9c28bdd58e


### get run details

In [25]:
run_name = "Titanic survived prediction 0.1.0"
run_id = None
runs = client.list_runs(experiment_id=experiment_id)

for run in runs.to_dict()['runs']:
    if run['display_name'] == run_name:
        run_id = run['run_id']
    
print(run_name, run_id)

Titanic survived prediction 0.1.0 None


### check run status

In [24]:
"""
{'run_id': 'd3998ea8-2be4-4f86-8598-85d46b0038c9',
   'task_id': 'b092fef4-11cd-49d3-b3ba-6052bd01f6fd',
   'display_name': 'iteration-item-driver',
   'create_time': datetime.datetime(2026, 1, 27, 9, 50, 29, tzinfo=tzlocal()),
   'start_time': datetime.datetime(2026, 1, 27, 9, 56, 15, tzinfo=tzlocal()),
   'end_time': datetime.datetime(2026, 1, 27, 9, 56, 18, tzinfo=tzlocal()),
   'executor_detail': None,
   'state': 'SUCCEEDED',
   'execution_id': None,
   'error': None,
   'inputs': None,
   'outputs': None,
   'parent_task_id': None,
"""
run = client.get_run(run_id=run_id)
run_details = run.run_details.to_dict()

for task in run_details['task_details']:
    if task["display_name"] and task["display_name"] in component_names:
        print(task["display_name"], task["state"])

autogluon-models-full-refit SUCCEEDED
autogluon-models-full-refit SUCCEEDED
train-test-split SUCCEEDED
automl-data-loader SUCCEEDED
autogluon-models-full-refit SUCCEEDED
for-loop-1 SUCCEEDED
models-selection SUCCEEDED
leaderboard-evaluation SUCCEEDED


<a id="display-leaderboard"></a>
## Display leaderboard

📝 **Note:** Ensure the S3 connection is added to the workbench so the notebook can access the results (e.g. `AWS_S3_ENDPOINT`, `AWS_S3_BUCKET`).


In [9]:
import boto3
import os
from IPython.display import HTML

s3 = boto3.resource('s3', endpoint_url=os.environ['AWS_S3_ENDPOINT'])
bucket = s3.Bucket(os.environ['AWS_S3_BUCKET'])
prefix = os.path.join(pipeline_name, run_id)
leaderboard_prefix = os.path.join(prefix, 'leaderboard-evaluation')
leaderboard_artifact_name = 'html_artifact'

for obj in bucket.objects.filter(Prefix=leaderboard_prefix):
    if leaderboard_artifact_name in obj.key:
        bucket.download_file(obj.key, leaderboard_artifact_name)

HTML(leaderboard_artifact_name)

,model,accuracy,balanced_accuracy,mcc,roc_auc,f1,precision,recall
0,NeuralNetFastAI_BAG_L1_FULL,0.903587,0.894618,0.798385,0.964869,0.876081,0.904762,0.849162
1,LightGBMXT_BAG_L2_FULL,0.903587,0.894618,0.798385,0.964869,0.876081,0.904762,0.849162
2,NeuralNetFastAI_BAG_L4_FULL,0.903587,0.894618,0.798385,0.964869,0.876081,0.904762,0.849162


<a id="download-trained-model"></a>
## Download trained model

📌 **Action:** Set `model_to_download` to the model you want to download (must match a name from the leaderboard **model** column).


In [10]:
model_to_download = "LightGBMXT_BAG_L2_FULL"

full_refit_prefix = os.path.join(prefix, "autogluon-models-full-refit")
best_model_subpath = os.path.join("model_artifact", model_to_download)
local_dir = None  # set to a path to download under a specific local directory
best_model_path = None

for obj in bucket.objects.filter(Prefix=full_refit_prefix):
    if best_model_subpath not in obj.key or obj.key.endswith("/"):
        continue
    target = (
        obj.key
        if local_dir is None
        else os.path.join(local_dir, os.path.relpath(obj.key, full_refit_prefix))
    )
    parent = os.path.dirname(target)
    if not os.path.exists(parent):
        os.makedirs(parent)
    bucket.download_file(obj.key, target)
    best_model_path = obj.key

if best_model_path is not None:
    best_model_path = os.path.join(
        best_model_path.split(model_to_download)[0], model_to_download
    )

<a id="load-the-model"></a>
## Load the model

Load the trained model as a TabularPredictor object.


In [11]:
%pip install autogluon.tabular==1.3.1 | tail -n 1

Note: you may need to restart the kernel to use updated packages.


In [12]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor.load(best_model_path, require_py_version_match=False)

/opt/app-root/lib64/python3.12/site-packages/autogluon/common/utils/utils.py:97: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Found 1 mismatches between original and current metadata:


### Score data

In [14]:
import pandas as pd

score_data = {
    "PassengerId": [1, 2], 
    "Pclass": [3, 1], 
    "Name": ["Braund, Mr. Owen Harris", "Heikkinen, Miss. Laina"], 
    "Sex": [0, 1],
    "Age": [22, 26],
    "SibSp": [1, 0],
    "Parch": [0, 0],
    "Ticket": ["A/5 21171", "STON/O2. 3101282"],
    "Fare": [7.25, 7.9],
    "Cabin": ["", ""],
    "Embarked": ["S", "C"]
}
score_df = pd.DataFrame(data=score_data)
score_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.25,,S
1,2,1,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.90,,C


### Predict the Survived value for score records

In [15]:
predictor.predict_proba(score_df)

,0,1
0,0.787024,0.212976
1,0.039443,0.960557


<a id="summary-and-next-steps"></a>
## Summary and next steps

**Summary:** This notebook monitored KFP run details, displayed the experiment leaderboard, downloaded a trained AutoGluon model from S3, and ran predictions on sample data.

**Next steps:**
- Use the predictor notebook to load and score from a different run (set `pipeline_name`, `run_id`, and `model_name`).
- Run predictions on your own data (ensure columns match the training schema).
- Optionally save the predictor locally with `predictor.save("path/to/dir")` for offline use.
